# 목차
    1. [라이브러리 불러오기](#라이브러리-불러오기)
    2. [파일 불러오기 & API 키 받아오기](#파일--api-키-받아오기)
    3. [gpt 필드 전처리](#gpt)
    4. [gpt 필드 정리](#생성-필드-합치고-저장하기)
    5. [필요없는 필드 제거 후 저장](#중복-필드-제거--저장)

# 라이브러리 불러오기

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pydantic import BaseModel
from typing import List, Optional

# 파일 & API 키 받아오기

In [33]:
# 환경변수에서 'OPENAPI_KEY' 값을 받아옴
# openapi_key = os.getenv('OPEN_API_KEY')

# GPT

In [96]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'preprocessing')
data_file_path = os.path.join(data_directory, 'processed_data.pkl')
df = pd.read_pickle(data_file_path)

In [97]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

In [98]:
df.drop(['expiration_date', 'market_price', 'capacity',
       'parking', 'options', 'check_in_out_time', 'shipping_fee',
       'transaction_location', 'transaction_method', 'city'],axis=1,inplace=True)

In [101]:
# 80글자 이상의 input만 필터링 -> 4974개
filtered_df = df[df['description'].str.len() >= 80]

In [102]:
from pydantic import BaseModel, Field
from typing import Optional
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np

# Pydantic을 사용해 응답 데이터를 구조화할 모델을 정의합니다.
class Fields(BaseModel):
    index: Optional[int] = Field(default=None)  # 인덱스를 Optional로 하고 기본값을 설정
    expiration_stdate: Optional[str]
    expiration_endate: Optional[str]
    room_type: Optional[str]
    head_count: Optional[int]
    shipping_fee: Optional[int]
    transaction_method: Optional[str]
    market_price: Optional[int]
    options: Optional[str]
    parking: Optional[bool]
    check_in_time: Optional[str]
    check_out_time: Optional[str]
    stay_type: Optional[str]
    foreign_yn: Optional[str]
    event: Optional[str]
    advertise: Optional[bool]

# 함수를 정의하여 데이터를 요청하고 응답을 처리합니다.
def extract_fields(description, index):
    prompt = {
        "role": "system", 
        "content": """
        너는 데이터 필드를 추출해주는 도우미야. 데이터를 받아서 필요한 필드를 15개로 제한해 JSON 형식으로 응답할거야. 
        설명에서 필요한 15개의 필드를 정확히 추출하고 나머지 필드들은 무시해줘. 필드는 오직 15개만 있어야 하고, 추가적인 세부 필드를 포함해서는 안 돼.
        NaN이나 정보가 없으면 해당 필드를 null로 설정해줘.
        필드는 아래 15개:
        1. expiration_stdate (해당 상품의 사용기한 중 시작 날짜 YYYY-MM-DD 형식, 만약 연도가 주어지지 않으면 post_time을 기준으로 연도를 추론해줘.)
        2. expiration_endate (해당 상품의 사용기한 중 마감 날짜 YYYY-MM-DD 형식, 만약 연도가 주어지지 않으면 post_time을 기준으로 연도를 추론해줘.)
        3. room_type (객실 유형)
        4. head_count (사용 인원수, int로만 알려줘)
        5. shipping_fee (배송비, 없으면 0)
        6. transaction_method (직거래/택배거래 여부, , 아래 규칙을 따름)
            규칙 : '직거래'라는 키워드가 있다면 '직거래', '택배거래'라는 키워드가 있다면 '택배거래', '직거래'와 '택배거래' 키워드가 모두 존재한다면 '둘 다 가능', 그 외의 모든 값들은 '기타'로 설정
        7. market_price (시중 가격, int로만 알려줘)
        8. options (호텔 옵션들 항목들을 콤마로 구분)
        9. parking (주차 가능 여부, True/False)
        10. check_in_time (체크인 시간, 형식 : 'HH:MM', 입력된 값이 HH:MM 형식이 아니면 null로 설정)
        11. check_out_time (체크아웃 시간, 형식 : 'HH:MM', 입력된 값이 HH:MM 형식이 아니면 null로 설정)
        12. stay_type (숙소 형태, 카테고리는 호텔, 모텔, 리조트, 풀빌라, 펜션, 카테고리 이외는 모두 기타로 분류)
        13. foreign_yn (국내 숙소인지 외국 숙소인지 알려줘, 국내/외국)
        14. event (축제/콘서트 등 관련 이벤트가 포함되어있으면 축제,콘서트 이름을 써줘)
        15. advertise (광고글 여부,숙박 시설을 판매 및 양도하는 경우는 광고글이 아니야. 대리 구매나 숙박이 아닌 다른 걸 판매하는게 광고글이야, True/False)"""
    }

    query = {
        "role": "user",
        "content": f"""
        설명에서 필요한 15개의 필드를 추출하고 JSON 형식으로 반환해줘. 단, 필드는 15개 고정이야.
        
        설명: {description}
        """
    }

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 "gpt-4" 사용 가능
            messages=[prompt, query],
            temperature=0,
            max_tokens=1000
        )

        # 응답을 받아 JSON으로 변환
        result = response['choices'][0]['message']['content']
        result = result.strip()  # 공백 제거

        # json으로 시작되는 쓸데없는 부분을 제거
        if result.startswith("```json"):
            result = result.replace("```json", "").strip()
        if result.endswith("```"):
            result = result.replace("```", "").strip()

        # Pydantic을 사용해 응답을 구조화된 데이터로 변환
        parsed_data = Fields.parse_raw(result)
        parsed_data.index = index  # 인덱스를 추가합니다.
        return parsed_data.dict()  # 딕셔너리로 반환

    except Exception as e:
        print(f"응답 파싱 중 오류 발생 (index: {index}):", e)
        return {field: np.nan for field in Fields.__fields__}  # NaN 값으로 채운 딕셔너리 반환

# 멀티쓰레딩과 배치 처리
def process_descriptions_in_batches(descriptions, batch_size=30):
    all_results = []
    for i in range(0, len(descriptions), batch_size):
        batch = descriptions[i:i+batch_size]
        batch_results = process_batch(batch, i)
        all_results.extend(batch_results)
    return all_results

# 배치 내에서 각 설명을 처리하는 함수
def process_batch(batch, start_index):
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_fields, desc, start_index + idx) for idx, desc in enumerate(batch)]
        
        # tqdm으로 진행률 표시
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing batch"):
            results.append(future.result())
    
    return results

# 결과를 데이터프레임으로 변환
def convert_to_dataframe(results):
    filtered_df = pd.DataFrame(results)
    return filtered_df

# 샘플 설명 데이터
descriptions = filtered_df['description_1'].tolist()

# 30개씩 배치로 처리
processed_data = process_descriptions_in_batches(descriptions, batch_size=30)

# DataFrame으로 변환
df_processed = convert_to_dataframe(processed_data)

processed_data

Processing batch:  43%|████▎     | 13/30 [00:02<00:01, 10.48it/s]

응답 파싱 중 오류 발생 (index: 1295): 1 validation error for Fields
room_type
  Input should be a valid string [type=string_type, input_value=['디럭스룸', '패밀리룸', '캐릭터룸'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


Processing batch:  67%|██████▋   | 20/30 [00:03<00:00, 11.51it/s]

응답 파싱 중 오류 발생 (index: 3759): 1 validation error for Fields
room_type
  Input should be a valid string [type=string_type, input_value=['Family Triple River Vie... View', 'Deluxe Double'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


Processing batch: 100%|██████████| 24/24 [00:05<00:00,  4.53it/s]


[{'index': 4,
  'expiration_stdate': '2024-10-11',
  'expiration_endate': '2024-11-05',
  'room_type': None,
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': None,
  'options': None,
  'parking': False,
  'check_in_time': None,
  'check_out_time': None,
  'stay_type': '호텔',
  'foreign_yn': '국내',
  'event': None,
  'advertise': False},
 {'index': 2,
  'expiration_stdate': '2025-03-01',
  'expiration_endate': '2025-03-03',
  'room_type': None,
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': 170000,
  'options': None,
  'parking': False,
  'check_in_time': None,
  'check_out_time': None,
  'stay_type': '호텔',
  'foreign_yn': '외국',
  'event': None,
  'advertise': False},
 {'index': 0,
  'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': '호텔',
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': None,
  'options': '무료 와인시음행사',
  'parking': False,
 

In [ ]:
df_processed

# 생성-필드-합치고-저장하기

In [117]:
# 다시 돌리는 일 없게 copy
up80_df=filtered_df.copy()
process_df = df_processed.copy()
df1=df.copy()

In [118]:
# Step 1: df_processed의 'index' 열을 인덱스로 설정
process_df.set_index('index', inplace=True)

# Step 2: filtered_df에 0~30까지의 새로운 인덱스를 부여
up80_df = up80_df.reset_index()  # 기존 인덱스를 제거하고 0부터 시작하는 새로운 인덱스 부여

# Step 3: 인덱스를 기준으로 두 DataFrame 병합
merge_df = pd.merge(up80_df, process_df, left_index=True, right_index=True, how='inner')
merge_df

,index,platform,original_link,post_time,title,view_count,like_count,price,images,description,...,transaction_method,market_price,options,parking,check_in_time,check_out_time,stay_type,foreign_yn,event,advertise
0.0,1,당근마켓,https://www.daangn.com/articles/846644988,2024-10-11 19:12:01.203000,용산역 호텔숙박권,15.0,1.0,280000.0,https://img.kr.gcp-karroter.net/origin/article...,용산역 4성급 호텔 이비스 스타일 앰버서더 숙박권입니다! \n날짜 내일 10월 *...,...,직거래,NaN,무료 와인시음행사,False,None,None,호텔,국내,와인시음행사,False
1.0,4,당근마켓,https://www.daangn.com/articles/846637088,2024-10-11 18:58:01.203000,대만 시먼딩 puri ximen 호텔 양도(3/1-3/3),6.0,1.0,150000.0,https://img.kr.gcp-karroter.net/origin/article...,대만 시먼딩에 위치한 puri ximen 호텔 양도합니다\n내년 연휴인 3/1-3/...,...,직거래,170000.0,None,False,None,None,호텔,외국,None,False
2.0,5,당근마켓,https://www.daangn.com/articles/846636618,2024-10-11 18:57:01.203000,대만 puri ximen 호텔 양도(3/1-3/3),20.0,1.0,150000.0,https://img.kr.gcp-karroter.net/origin/article...,대만 시먼딩에 위치한 puri ximen 호텔 양도합니다\n내년 연휴인 3/1-3/...,...,직거래,170000.0,None,False,None,None,호텔,외국,None,False
3.0,7,당근마켓,https://www.daangn.com/articles/846631628,2024-10-11 18:48:01.203000,[팝니다] 울산 삼산 롯데시티 호텔 숙박 이용권,25.0,0.0,100000.0,정보없음,울산 삼삼동에 위치한 롯데시티 호텔 숙박 이용권 판매합니다.\n일자 : 11(금) ...,...,직거래,181500.0,None,False,None,None,호텔,국내,None,False
4.0,8,당근마켓,https://www.daangn.com/articles/846629935,2024-10-11 18:45:01.203000,제네시스 프리빌리지 파라다이스호텔,15.0,0.0,300000.0,정보없음,제네시스 프리빌리지 (부산 파라다이스호텔)\n양도합니다.\n2024년 11월5일까지...,...,직거래,NaN,None,False,None,None,호텔,국내,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969.0,7820,중고나라,https://web.joongna.com/product/177788182,2024-07-24 13:18:03,7월28일~31일 한화리조트 여수벨메르호텔 양도합니다,6.0,0.0,12345.0,https://img2.joongna.com/cafe-article-data/liv...,"※ ※한화리조트 여수 벨메르호텔 미취사 침구추가 2채까지, 1채 30.000원 7월...",...,직거래,690000.0,미취사 침구추가 2채까지,False,None,None,리조트,국내,None,False
4970.0,7822,중고나라,https://web.joongna.com/product/177776248,2024-07-24 11:52:53,7월26일 하이원리조트 강원랜드 그랜드호텔 메인타워 트리플 있습니다,18.0,0.0,220000.0,https://img2.joongna.com/cafe-article-data/liv...,7월26일 하이원리조트 그랜드호텔 메인타워 트리플 22만원 연락주세요 010-904...,...,직거래,22000.0,워터월드,False,None,None,리조트,국내,None,False
4971.0,7823,중고나라,https://web.joongna.com/product/177775606,2024-07-24 11:48:14,"강릉 세인트존스호텔 양도 7월 24, 25, 26, 27, 28",17.0,0.0,200000.0,https://img2.joongna.com/cafe-article-data/liv...,강릉 세인트존스호텔 7/24 20만원 7/25 20만원 7/26 24만원 7/27 ...,...,직거래,NaN,조식,False,None,None,호텔,국내,None,False
4972.0,7824,중고나라,https://web.joongna.com/product/177776021,2024-07-24 11:51:09,잠실 라세느 호텔 뷔페 식사권 2매​주말 저녁 2타임 8시에 사용 가능합니다. 2장...,33.0,0.0,150000.0,https://img2.joongna.com/cafe-article-data/liv...,잠실 라세느 호텔 뷔페 식사권 2매 주말 저녁 2타임 8시에 사용 가능합니다. 2장...,...,직거래,NaN,None,False,None,None,호텔,국내,None,False


In [119]:
# 1. filtered_merged_df에서 인덱스를 기준으로 병합
df1 = df1.merge(merge_df, left_index=True, right_on='index', how='left')
df1

,platform_x,original_link_x,post_time_x,title_x,view_count_x,like_count_x,price_x,images_x,description_x,category_x,...,transaction_method,market_price,options,parking,check_in_time,check_out_time,stay_type,foreign_yn,event,advertise
NaN,당근마켓,https://www.daangn.com/articles/846645149,2024-10-11 19:12:01.203000,호텔숙바퀀,25.0,0.0,10000.0,https://img.kr.gcp-karroter.net/origin/article...,6시입실 내일12시퇴실\n가려했는데같이가는사람이 시간이 안맞아서요~,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,당근마켓,https://www.daangn.com/articles/846644988,2024-10-11 19:12:01.203000,용산역 호텔숙박권,15.0,1.0,280000.0,https://img.kr.gcp-karroter.net/origin/article...,용산역 4성급 호텔 이비스 스타일 앰버서더 숙박권입니다! \n날짜 내일 10월 *...,티켓/교환권,...,직거래,NaN,무료 와인시음행사,False,None,None,호텔,국내,와인시음행사,False
NaN,당근마켓,https://www.daangn.com/articles/846639021,2024-10-11 19:01:01.203000,호텔 아쿠아펠리스 숙박권,79.0,2.0,100000.0,https://img.kr.gcp-karroter.net/origin/article...,사정이생겨 못가게되서\n선착순한명 싸게드릴께요,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,당근마켓,https://www.daangn.com/articles/846637216,2024-10-11 18:58:01.203000,워커힐호텔 피자힐,39.0,2.0,90000.0,https://img.kr.gcp-karroter.net/origin/article...,피자힐 10만원권 9만원에 판매합니다~!,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,당근마켓,https://www.daangn.com/articles/846637088,2024-10-11 18:58:01.203000,대만 시먼딩 puri ximen 호텔 양도(3/1-3/3),6.0,1.0,150000.0,https://img.kr.gcp-karroter.net/origin/article...,대만 시먼딩에 위치한 puri ximen 호텔 양도합니다\n내년 연휴인 3/1-3/...,티켓/교환권,...,직거래,170000.0,None,False,None,None,호텔,외국,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,중고나라,https://web.joongna.com/product/177764839,2024-07-24 10:39:59,롯데호텔 시그니엘서울 애프터눈티 세트 16>13,20.0,0.0,130000.0,https://img2.joongna.com/cafe-article-data/liv...,16>13,기타 티켓/쿠폰/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,중고나라,https://web.joongna.com/product/177767396,2024-07-24 10:55:25,영등포 리더스 호텔(모텔)7/27-28디락스 트윈룸 숙박권 양도합니다,28.0,0.0,70000.0,/cafe-article-data/live/2024/07/24/1067914533/...,일정에 갑자기 차질이 생겨서 숙박권 판매 하려고 합니다 혹시 생각 있으신 분은 연락...,여행숙박/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,중고나라,https://web.joongna.com/product/177766185,2024-07-24 10:47:40,롯데호텔통합숙박권,93.0,2.0,300000.0,https://img2.joongna.com/cafe-article-data/liv...,롯데호텔통합숙박권입니다 사용처 및 구성은 사진에 나와있습니다 24년 12월 말까지 ...,여행숙박/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,중고나라,https://web.joongna.com/product/177758738,2024-07-24 09:59:22,씨마크 호텔이용권 (11만원),22.0,1.0,85000.0,https://img2.joongna.com/cafe-article-data/liv...,호텔이용권입니다 11만원 이용권 24.8.31 까지,여행숙박/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
df1.columns

Index(['platform_x', 'original_link_x', 'post_time_x', 'title_x',
       'view_count_x', 'like_count_x', 'price_x', 'images_x', 'description_x',
       'category_x', 'status_x', 'seller_location_x', 'city_goo_x',
       'city_dong_x', 'description_1_x', 'index', 'platform_y',
       'original_link_y', 'post_time_y', 'title_y', 'view_count_y',
       'like_count_y', 'price_y', 'images_y', 'description_y', 'category_y',
       'status_y', 'seller_location_y', 'city_goo_y', 'city_dong_y',
       'description_1_y', 'expiration_stdate', 'expiration_endate',
       'room_type', 'head_count', 'shipping_fee', 'transaction_method',
       'market_price', 'options', 'parking', 'check_in_time', 'check_out_time',
       'stay_type', 'foreign_yn', 'event', 'advertise'],
      dtype='object')

In [126]:
df1.drop(['platform_y', 'original_link_y', 'post_time_y', 'title_y', 'view_count_y', 'like_count_y', 'price_y', 'images_y', 'description_y', 'category_y',
       'status_y', 'seller_location_y', 'city_goo_y', 'city_dong_y', 'description_1_y'],axis=1,inplace=True)
df1.columns

Index(['platform_x', 'original_link_x', 'post_time_x', 'title_x',
       'view_count_x', 'like_count_x', 'price_x', 'images_x', 'description_x',
       'category_x', 'status_x', 'seller_location_x', 'city_goo_x',
       'city_dong_x', 'description_1_x', 'index', 'expiration_stdate',
       'expiration_endate', 'room_type', 'head_count', 'shipping_fee',
       'transaction_method', 'market_price', 'options', 'parking',
       'check_in_time', 'check_out_time', 'stay_type', 'foreign_yn', 'event',
       'advertise'],
      dtype='object')

# 중복 필드 제거 & 저장

In [127]:
# `_x`가 붙은 열의 이름에서 `_x`를 제거
df1.columns = df1.columns.str.replace('_x$', '', regex=True)

# 'index' 열을 인덱스로 설정
df1.set_index('index', inplace=True)

# 결과 확인
df1

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,...,transaction_method,market_price,options,parking,check_in_time,check_out_time,stay_type,foreign_yn,event,advertise
index,,,,,,,,,,,,,,,,,,,,,
0,당근마켓,https://www.daangn.com/articles/846645149,2024-10-11 19:12:01.203000,호텔숙바퀀,25.0,0.0,10000.0,https://img.kr.gcp-karroter.net/origin/article...,6시입실 내일12시퇴실\n가려했는데같이가는사람이 시간이 안맞아서요~,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,당근마켓,https://www.daangn.com/articles/846644988,2024-10-11 19:12:01.203000,용산역 호텔숙박권,15.0,1.0,280000.0,https://img.kr.gcp-karroter.net/origin/article...,용산역 4성급 호텔 이비스 스타일 앰버서더 숙박권입니다! \n날짜 내일 10월 *...,티켓/교환권,...,직거래,NaN,무료 와인시음행사,False,None,None,호텔,국내,와인시음행사,False
2,당근마켓,https://www.daangn.com/articles/846639021,2024-10-11 19:01:01.203000,호텔 아쿠아펠리스 숙박권,79.0,2.0,100000.0,https://img.kr.gcp-karroter.net/origin/article...,사정이생겨 못가게되서\n선착순한명 싸게드릴께요,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,당근마켓,https://www.daangn.com/articles/846637216,2024-10-11 18:58:01.203000,워커힐호텔 피자힐,39.0,2.0,90000.0,https://img.kr.gcp-karroter.net/origin/article...,피자힐 10만원권 9만원에 판매합니다~!,티켓/교환권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,당근마켓,https://www.daangn.com/articles/846637088,2024-10-11 18:58:01.203000,대만 시먼딩 puri ximen 호텔 양도(3/1-3/3),6.0,1.0,150000.0,https://img.kr.gcp-karroter.net/origin/article...,대만 시먼딩에 위치한 puri ximen 호텔 양도합니다\n내년 연휴인 3/1-3/...,티켓/교환권,...,직거래,170000.0,None,False,None,None,호텔,외국,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,중고나라,https://web.joongna.com/product/177764839,2024-07-24 10:39:59,롯데호텔 시그니엘서울 애프터눈티 세트 16>13,20.0,0.0,130000.0,https://img2.joongna.com/cafe-article-data/liv...,16>13,기타 티켓/쿠폰/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7827,중고나라,https://web.joongna.com/product/177767396,2024-07-24 10:55:25,영등포 리더스 호텔(모텔)7/27-28디락스 트윈룸 숙박권 양도합니다,28.0,0.0,70000.0,/cafe-article-data/live/2024/07/24/1067914533/...,일정에 갑자기 차질이 생겨서 숙박권 판매 하려고 합니다 혹시 생각 있으신 분은 연락...,여행숙박/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7828,중고나라,https://web.joongna.com/product/177766185,2024-07-24 10:47:40,롯데호텔통합숙박권,93.0,2.0,300000.0,https://img2.joongna.com/cafe-article-data/liv...,롯데호텔통합숙박권입니다 사용처 및 구성은 사진에 나와있습니다 24년 12월 말까지 ...,여행숙박/이용권,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 혹시 몰라서 생성 필드만 저장한 파일
df_processed.to_csv("df_processed.csv",encoding='utf-8-sig')

In [130]:
# 저장
df1.to_csv('final_data_2024_10_15.csv',encoding='utf-8-sig')